In [15]:
import pandas as pd
from itertools import combinations
from collections import Counter
import ast 

In [10]:
data = pd.read_csv("games_full.csv")
data.head()

,title,gameid,developers,publishers,genres,supported_languages,release_date,PS,Steam,xBox
0,Fieldrunners 2,215710,['Subatomic Studios LLC'],['Subatomic Studios LLC'],"['Action', 'Strategy', 'Casual']",['English'],2013-01-10,NaN,Yes,NaN
1,Goddess detective 2,2475270,['2024-NGame'],['2024-NGame'],['Indie'],"['English', 'Russian', 'Simplified Chinese', '...",2023-08-31,NaN,Yes,NaN
2,Magic pal and great,3203600,['神奇伙伴与大圣'],['神奇伙伴与大圣'],"['Action', 'Adventure', 'Casual', 'RPG']","['English', 'Simplified Chinese']",2024-10-19,NaN,Yes,NaN
3,Promise with My Sister,2843130,['Carnival Pepper'],['Carnival Pepper'],"['Casual', 'Indie', 'RPG', 'Strategy']","['Japanese', 'Simplified Chinese', 'Traditiona...",2024-05-09,NaN,Yes,NaN
4,Sakura no Mori † Dreamers 2,983150,['MOONSTONE'],['HIKARI FIELD'],['Adventure'],"['Simplified Chinese', 'Traditional Chinese']",2019-03-15,NaN,Yes,NaN


In [11]:
df = pd.read_csv('PSpurchased_games.csv')
df.head()

,playerid,library
0,268071,"[14972, 417905, 14693, 7742, 20162, 461983, 12..."
1,2218485,"[410618, 17456, 16034, 138931, 175202, 331007,..."
2,253885,"[417808, 706747, 619583, 492711, 555631, 18167..."
3,1911732,"[670209, 694794, 658551, 582894, 602666, 65531..."
4,3098524,"[19362, 170350, 12653, 20230, 12750, 18141, 41..."


In [16]:
# convert stringified lists to actual lists
df['library'] = df['library'].apply(ast.literal_eval)

# compute game pair occurrences
pair_counts = Counter()

for games in df['library']:
    if isinstance(games, list) and len(games) > 1:
        for game1, game2 in combinations(sorted(games), 2):
            pair_counts[(game1, game2)] += 1

# create df
pair_df = pd.DataFrame([(g1, g2, count) for (g1, g2), count in pair_counts.items()],
                       columns=['game1', 'game2', 'pair_count'])

# calculate total occurrences of each game
game_counts = df['library'].explode().astype(int).value_counts().to_dict()

# calculate pair percentage
def calc_percentage(row):
    game1_count = game_counts.get(row['game1'], 0)
    return row['pair_count'] / game1_count if game1_count > 0 else 0

pair_df['pair_percentage'] = pair_df.apply(calc_percentage, axis=1)

# Sort by pair_count
pair_df = pair_df.sort_values('pair_count', ascending=False).reset_index(drop=True)

print(pair_df.head())

    game1   game2  pair_count  pair_percentage
0    7779    9459       24282         0.790404
1   16508  137118       22522         0.847616
2    7779  333860       22034         0.717229
3    9459  333860       21715         0.726254
4  137118  163258       21281         0.739771


In [18]:
pair_df.shape

(239325533, 4)